## RAG with Multiple sources

'https://python.langchain.com/v0.1/docs/modules/tools/'

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper=WikipediaAPIWrapper(top_k_results=2,doc_content_chars_max=500)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OllamaEmbeddings(model="llama3"))
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024DB7938FB0>)

In [6]:
#custom LangSmith retriever tool
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [7]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)


In [10]:
'sequence of tools is what the llm will use to search first, if noy found ,next'
tools=[wiki,arxiv,retriever_tool]
tools


[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'C:\\TUM\\Python_LangChain\\ChatBot\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=2, lang='en', load_all_available_meta=False, doc_content_chars_max=500)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='Search information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000024D98B0D940>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], ve

In [46]:
import os
from langchain_community.chat_models import ChatPerplexity
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
load_dotenv()
os.environ["PPLX_API_KEY"] = os.getenv("PPLX_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
llm = ChatPerplexity(model="llama-3-sonar-large-32k-online")
output_parser=StrOutputParser()

In [47]:
from langchain import hub
# Get the prompt to use from lots of templates available
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [55]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [58]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
chain=agent_executor | output_parser
chain

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [63]:
agent_executor.invoke({"input":"What is Dieter Schwarz Stiftung?"})




> Entering new AgentExecutor chain...
Dieter Schwarz Stiftung, also known as the Dieter Schwarz Foundation, is a non-profit charitable foundation established by German entrepreneur Dieter Schwarz in 1999. Based in Heilbronn, the foundation focuses on promoting education and science, particularly in the fields of lifelong learning, with a guiding principle of "Promote education, share knowledge, venture future." It supports various initiatives and institutions, including schools, research institutions, and start-up activities, aiming to identify potential, develop visions, and implement them sustainably. The foundation is active where government bodies are not, or not sufficiently, able to meet the needs of the economy and society.

> Finished chain.


{'input': 'What is Dieter Schwarz Stiftung?',
 'output': 'Dieter Schwarz Stiftung, also known as the Dieter Schwarz Foundation, is a non-profit charitable foundation established by German entrepreneur Dieter Schwarz in 1999. Based in Heilbronn, the foundation focuses on promoting education and science, particularly in the fields of lifelong learning, with a guiding principle of "Promote education, share knowledge, venture future." It supports various initiatives and institutions, including schools, research institutions, and start-up activities, aiming to identify potential, develop visions, and implement them sustainably. The foundation is active where government bodies are not, or not sufficiently, able to meet the needs of the economy and society.'}

In [65]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about? explain some details about it?"})



> Entering new AgentExecutor chain...
The paper "1605.08386" is titled "Heat-bath random walks with Markov bases" and is authored by Caprice Stanley and Tobias Windisch. It was published on May 26, 2016. The paper discusses the study of graphs on lattice points with edges coming from a finite set of allowed moves. It explores the diameter of these graphs and the mixing behavior of heat-bath random walks on them. The paper also discusses conditions for the heat-bath random walk to be an expander in fixed dimension.

> Finished chain.


{'input': "What's the paper 1605.08386 about? explain some details about it?",
 'output': 'The paper "1605.08386" is titled "Heat-bath random walks with Markov bases" and is authored by Caprice Stanley and Tobias Windisch. It was published on May 26, 2016. The paper discusses the study of graphs on lattice points with edges coming from a finite set of allowed moves. It explores the diameter of these graphs and the mixing behavior of heat-bath random walks on them. The paper also discusses conditions for the heat-bath random walk to be an expander in fixed dimension.'}